In [1]:
# Dependencies
import requests
import pandas as pd
from config import api_key

In [15]:
# Save config information.
#https://openweathermap.org/current#zip --> INFO OF THE API HERE
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
#query_url = f"{url}appid={api_key}&units={units}&q="
query_url = f"{url}appid={api_key}&units={units}&zip="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=1e44bc9d31c9878cf91aa040278c13af&units=metric&zip=


In [5]:
zip_data = pd.read_csv("zips_output.csv")
zip_data.head()

,us_zip_code,type,primary_city,state,latitude,longitude,county,timezone,area_codes,country,estmiated_population,created_at,updated_at
0,501,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631.0,US,384,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
1,602,STANDARD,Aguada,PR,18.38,-67.18,NaN,NaN,787.0,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
2,601,STANDARD,Adjuntas,PR,18.16,-66.72,Adjuntas,America/Puerto_Rico,787939.0,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
3,544,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631.0,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
4,603,STANDARD,Aguadilla,PR,18.43,-67.15,Aguadilla,America/Puerto_Rico,787.0,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z


In [19]:
zip_codes= zip_data["us_zip_code"]
updated_zip_codes = []
for zip_code in zip_codes:
    if len(str(zip_code)) == 3:
        updated_zip_codes.append(f'00{zip_code}')
    elif len(str(zip_code)) == 4:
        updated_zip_codes.append(f'0{zip_code}')
    else:
        updated_zip_codes.append(zip_code)

In [24]:
zip_codes= zip_data["us_zip_code"]

# set up lists to hold reponse info
zip_codes_final = []
lat = []
lng = []
temp = []
pressure = []
humidity = []
wind_speed = []
clouds = []

# Loop through the list of cities and perform a request for data on each
for zip_code in updated_zip_codes:
    try:
        response = requests.get(query_url + zip_code).json()
        zip_codes_final.append(zip_code)
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        pressure.append(response['main']['pressure'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        clouds.append(response['clouds']['all'])
    except:
        print(f"{zip_code} doesn't exist")


In [25]:
# create a data frame from cities, lat, and temp
# create a data frame from cities, lat, and temp
weather_dict = {
    "zip_code": zip_codes_final,
    "lat": lat,
    "lng": lng,
    "temp": temp,
    "pressure": pressure,
    "humidity" : humidity,
    "wind_speed" : wind_speed,
    "clouds" : clouds
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,zip_code,lat,lng,temp,pressure,humidity,wind_speed,clouds
0,00501,40.8154,-73.0451,7.07,1020,48,6.69,20
1,00602,18.3800,-67.1800,27.57,1016,72,2.68,10
2,00601,18.1600,-66.7200,25.97,1017,65,3.36,24
3,00544,40.8154,-73.0451,7.07,1020,48,6.69,20
4,00603,18.4300,-67.1500,27.20,1016,74,2.68,11
...,...,...,...,...,...,...,...,...
995,03037,43.1378,-71.2513,5.90,1012,43,0.89,5
996,03045,43.0201,-71.5697,6.51,1013,36,5.81,6
997,03042,43.0411,-71.0764,6.65,1012,40,2.24,5
998,03043,42.9920,-71.8113,5.49,1013,36,0.89,5
